In [1]:
# Core
import numpy as np
import pandas as pd

import time

In [2]:
# Local imports
import kepler_sieve
from asteroid_skypatch import calc_ast_skypatch, test_skypatch_dir
from asteroid_direction import ast, c, mpd, dpm, prep_ast_block, calc_dir_ast_block
from asteroid_spline import make_spline_ast_dir
from sky_patch import dir2SkyPatchID, SkyPatchID2dir
from db_utils import sp2df, df2db

# Build block of asteroid directions at high resolution

In [3]:
# Asteroid block
n0: int = 1
n1: int = 100

# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
interval_min: int = 15

In [4]:
# Asteroid block
n0: int = 1
n1: int = 3

# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
interval_min: int = 15

In [5]:
t0 = time.time()
df = calc_ast_skypatch(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, interval_min=interval_min)
t1 = time.time()

In [6]:
et = t1 - t0
print(f'Elapsed time with interval={interval_min}: {et:5.2f}')

Elapsed time with interval=15:  0.42


In [7]:
df

,AsteroidID,Segment,SkyPatchID,TimeID_0,TimeID_1
0,1,0,6382172,83520000,83520135
1,1,1,6382171,83520135,83520225
2,1,2,6384219,83520225,83520315
3,1,3,6384218,83520315,83520495
4,1,4,6384217,83520495,83520675
...,...,...,...,...,...
33092,2,18884,4488003,86399550,86399670
33093,2,18885,4490051,86399670,86399700
33094,2,18886,4490050,86399700,86399790
33095,2,18887,4492098,86399790,86399910


# Test direction at midpoint of interval vs. center of sky patch

In [8]:
# Time at the midpoint of each interval
t_obs_0 = df.TimeID_0.values / mpd
t_obs_1 = df.TimeID_1.values / mpd
t_obs = (t_obs_0 + t_obs_1)/2.0

In [9]:
df['tObs'] = t_obs

In [10]:
# Extract arrays
asteroid_id = df.AsteroidID.values
sky_patch_id = df.SkyPatchID.values

In [11]:
# # Direction used to build sky patch DataFrame
# cols_u = ['ux', 'uy', 'uz']
# u_df = df[cols_u].values

In [12]:
# Spline for asteroid directions vs. time
spline_u = make_spline_ast_dir(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [13]:
# Evaluate splined asteroid directions
u_ast, lt_ast = spline_u(ts=t_obs, asteroid_id=asteroid_id)

In [14]:
# Direction at the center of each sky patch
N_sp = 1024
u_sky = SkyPatchID2dir(SkyPatchID=sky_patch_id, N=N_sp)

In [15]:
# Compare splined asteroid direction to center of skypatch
du = test_skypatch_dir(name1='spline', name2='skypatch', u1=u_ast, u2=u_sky, verbose=True)

Angle Difference: skypatch vs. spline in arc seconds
*Mean  :      48.9*
 Median:      45.9
 Max   :     144.4
